In [ ]:
from dataclasses import dataclass, field
from typing import Any, Dict, Iterator, MutableMapping
from tklearn.nn.base import ModelConfig

class SampleConfig(ModelConfig):
    name: str
    value: int
    options: Dict[str, Any] = field(default_factory=dict)


# Example usage
config = SampleConfig(name="test", value=42, options={"debug": True})
print(config["name"])  # Accessing like a dict
config["value"] = 100  # Setting like a dict
print("name" in config)  # Checking if key exists
print(config.get("options"))  # Using get method
print(config.to_dict())  # Converting to dictionary
print(len(config))  # Getting the number of items
del config["options"]  # Deleting an item
print(list(config))  # Iterating over keys


In [ ]:
@dataclass
class Book:
    title: str

    def __init__(self, config) -> None:
        print(config)
        self.config = config

    @property
    def title(self) -> str:
        return self._title

    @title.setter
    def title(self, value: str):
        self._title = value

In [ ]:
book = Book(100)

In [ ]:
book.title

In [ ]:
from dataclasses import dataclass, asdict, fields

@dataclass
class User:
    name: str

@dataclass
class Project:
    name: str
    owner: User

    def __setattr__(self, __name, __value):
        field_types = {field.name: field.type for field in fields(self)}
        __type = field_types[__name]
        print(__name, __value, __type)
        if not isinstance(__value, __type):
            __value = __type(**__value)
        return super().__setattr__(__name, __value)

In [ ]:
asdict(Project("Kraken", User("John Doe")))

In [ ]:
Project(**{"name": "kraken", "owner": {"name": "John Doe"}})

In [ ]:
from dataclasses import dataclass
from datetime import datetime
from pathlib import Path
from typing import List, Union

@dataclass
class Person:
    name: str
    age: int
    birthdate: datetime
    addresses: List[Path]
    nickname: Union[str, None]

data = {
    "name": "John Doe",
    "age": 30,
    "birthdate": "2000-01-01T00:00:00",
    "addresses": ["/home/john", "/work/john"],
    "nickname": None
}

person = to_type(data, Person)
print(person)

In [ ]:
import copy
import sys
from collections.abc import Mapping
from dataclasses import fields, is_dataclass
from datetime import date, datetime, time
from pathlib import Path
from typing import (
    Any,
    ForwardRef,
    Generic,
    List,
    Tuple,
    Type,
    TypeVar,
    Union,
    get_args,
    get_origin,
    get_type_hints,
)

T = TypeVar("T")


def _get_type_hints(cls, globalns: Any = None, localns: Any = None):
    types = {}
    hints = get_type_hints(cls, globalns=globalns, localns=localns)
    for field in fields(cls):
        if field.name not in hints:
            continue
        types[field.name] = hints[field.name]
    return types


def evaluate_forwardref(typ: ForwardRef, globalns: Any, localns: Any) -> Any:
    if sys.version_info < (3, 9):
        return typ._evaluate(globalns, localns)
    if sys.version_info < (3, 12, 4):
        return typ._evaluate(globalns, localns, set())
    # fixing error: TypeError: ForwardRef._evaluate() missing 1 required
    # keyword-only argument: 'recursive_guard'
    return cast(Any, typ)._evaluate(
        globalns, localns, set(), recursive_guard=set()
    )


def asdict(obj, dispatch: bool = True):
    return _asdict_inner(obj, dict, dispatch=dispatch)


def _asdict_inner(obj, dict_factory, dispatch: bool = True):
    if dispatch and isinstance(obj, Dispatch):
        return obj.__class__._registry.dump(obj)
    if is_dataclass(obj):
        result = []
        for fn in _get_type_hints(obj.__class__):
            value = _asdict_inner(getattr(obj, fn), dict_factory)
            result.append((fn, value))
        return dict_factory(result)
    elif isinstance(obj, tuple) and hasattr(obj, "_fields"):
        return type(obj)(*[_asdict_inner(v, dict_factory) for v in obj])
    elif isinstance(obj, (list, tuple)):
        return type(obj)(_asdict_inner(v, dict_factory) for v in obj)
    elif isinstance(obj, dict):
        return type(obj)(
            (_asdict_inner(k, dict_factory), _asdict_inner(v, dict_factory))
            for k, v in obj.items()
        )
    else:
        return copy.deepcopy(obj)


def cast(
    typ: Type[T], val: Any, globalns: Any = None, localns: Any = None
) -> T:
    if globalns is None:
        globalns = globals()
    if localns is None:
        localns = {}
    # Handle Any type
    if typ is Any:
        return val
    # Handle Union types
    type_args = get_args(typ)
    origin = get_origin(typ)
    if origin is not None:
        typ = origin
    if isinstance(typ, str):
        typ = eval(typ, globalns, localns)  # noqa: S307
    if isinstance(typ, ForwardRef):
        typ = evaluate_forwardref(typ, globalns=globalns, localns=localns)
    if typ is Union:
        for subtype in type_args:
            try:
                return cast(subtype, val)
            except ValueError:
                continue
        msg = f"could not convert to any type in Union: {typ}"
        raise ValueError(msg)
    if issubclass(typ, Dispatch):
        return typ._registry.load(val)
    # Handle basic types
    if issubclass(typ, (int, float, str, bool)):
        return typ(val)
    # Handle datetime types
    if issubclass(typ, datetime):
        return datetime.fromisoformat(val)
    if issubclass(typ, date):
        return date.fromisoformat(val)
    if issubclass(typ, time):
        return time.fromisoformat(val)
    # Handle Path
    if issubclass(typ, Path):
        return Path(val)
    # Handle dataclasses
    if is_dataclass(typ):
        if isinstance(val, typ):
            return val
        if isinstance(val, dict):
            field_types = _get_type_hints(typ)
            kwargs = {
                k: cast(
                    field_types.get(k, Any),
                    v,
                    globalns=globalns,
                    localns=localns,
                )
                for k, v in val.items()
            }
            return typ(**kwargs)
        msg = f"could not convert to dataclass: {typ}"
        raise ValueError(msg)
    # Handle tuples
    if issubclass(typ, Tuple):
        try:
            if len(type_args) > 0:
                itype = Any
                if type_args[-1] is Ellipsis:
                    itype = type_args[-2]
                tuple_vals = []
                for i, item in enumerate(val):
                    tuple_vals.append(
                        cast(
                            type_args[i]
                            if i < len(type_args)
                            and type_args[i] is not Ellipsis
                            else itype,
                            item,
                            globalns=globalns,
                            localns=localns,
                        )
                    )
                return tuple(tuple_vals)
            return typ(
                cast(Any, item, globalns=globalns, localns=localns)
                for item in val
            )
        except ValueError:
            msg = f"could not convert to tuple: {typ}"
            raise ValueError(msg) from None
    # Handle lists
    if issubclass(typ, List):
        try:
            itype = type_args[0]
            return [
                cast(itype, item, globalns=globalns, localns=localns)
                for item in val
            ]
        except IndexError:
            return [
                cast(Any, item, globalns=globalns, localns=localns)
                for item in val
            ]
    # Handle dictionaries
    if issubclass(typ, Mapping):
        if not isinstance(val, dict):
            msg = f"could not convert to dict of type {typ}"
            raise ValueError(msg)
        ktype, vtype = Any, Any
        if len(type_args) == 2:
            ktype, vtype = type_args
        return {
            cast(ktype, k, globalns=globalns, localns=localns): cast(
                vtype, v, globalns=globalns, localns=localns
            )
            for k, v in dict(val).items()
        }
    msg = f"could not convert to type: {typ}"
    raise ValueError(msg)


class DispatchRegistry(Generic[T]):
    def __init__(self, base: T, attrs: List[str]):
        self.types = {}
        self.attrs = attrs

    def register(self, cls):
        self.types[tuple(getattr(cls, a) for a in self.attrs)] = cls

    def load(self, data: dict) -> T:
        data = data.copy()
        idv = []
        for a in self.attrs:
            idv.append(data.pop(a))
        idv = tuple(idv)
        return self.types[idv](**data)

    def dump(self, obj: Any) -> dict:
        data = asdict(obj, dispatch=False)
        for a in self.attrs:
            data[a] = getattr(obj, a)
        return data


class Dispatch(Generic[T]):
    _registry: DispatchRegistry

    def __init_subclass__(cls, **kwargs):
        super().__init_subclass__()
        if not hasattr(cls, "_dispatch") and "dispatch" in kwargs:
            cls._registry = DispatchRegistry(cls, kwargs.get("dispatch", []))
        else:
            cls._registry.register(cls)


In [ ]:
@dataclass
class Database(Dispatch, dispatch=["db"]):
    db: ClassVar[str]


@dataclass
class MySQL(Database):
    db: ClassVar[str] = "mysql"
    host: str
    port: int


@dataclass
class PostgreSQL(Database):
    db: ClassVar[str] = "postgresql"
    host: str
    port: int


@dataclass
class Project:
    name: str
    db: Database

In [ ]:
data = asdict(MySQL("localhost", 3306))
print(data)
obj = cast(Database, data)
assert isinstance(obj, MySQL)

In [ ]:
data = asdict(PostgreSQL("localhost", 5432))
print(data)
obj = cast(Database, data)
assert isinstance(obj, PostgreSQL)

In [ ]:
project = Project("test", MySQL("localhost", 3306))

project_data = asdict(project)

print(project_data)

project = cast(Project, project_data)

project

In [ ]:
project = Project("test", PostgreSQL("localhost", 5432))

project_data = asdict(project)

print(project_data)

project = cast(Project, project_data)

project